In [8]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

In [9]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
# making class names shorter
class_names = [x.split('.')[-1] if 'misc' not in x else '.'.join(x.split('.')[-2:]) for x in newsgroups_train.target_names]
class_names[3] = 'pc.hardware'
class_names[4] = 'mac.hardware'

In [10]:
print(','.join(class_names))

atheism,graphics,ms-windows.misc,pc.hardware,mac.hardware,x,misc.forsale,autos,motorcycles,baseball,hockey,crypt,electronics,med,space,christian,guns,mideast,politics.misc,religion.misc


In [11]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [12]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)
nb.fit(train_vectors, newsgroups_train.target)

MultinomialNB(alpha=0.01)

In [27]:
pred = nb.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='weighted')

0.8350184193998174

In [28]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, nb)

In [29]:
print(c.predict_proba([newsgroups_test.data[0]]).round(3))

[[0.001 0.01  0.003 0.047 0.006 0.002 0.003 0.521 0.022 0.008 0.025 0.
  0.331 0.003 0.006 0.    0.003 0.    0.001 0.009]]


In [39]:
vectorizer.transform([newsgroups_test.data[0]])

<1x95 sparse matrix of type '<class 'numpy.float64'>'
	with 95 stored elements in Compressed Sparse Row format>

In [17]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [18]:
idx = 1340
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6, labels=[0, 17])
print('Document id: %d' % idx)
print('Predicted class =', class_names[nb.predict(test_vectors[idx]).reshape(1,-1)[0,0]])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

Document id: 1340
Predicted class = atheism
True class: atheism


In [19]:
newsgroups_test.data[idx]

'From: conor@owlnet.rice.edu (Conor Frederick Prischmann)\nSubject: Re: Genocide is Caused by Theism : Evidence?\nOrganization: Rice University\nLines: 23\n\nIn article <C60A0s.DvI@mailer.cc.fsu.edu> dekorte@dirac.scri.fsu.edu (Stephen L. DeKorte) writes:\n>\n>I saw a 3 hour show on PBS the other day about the history of the\n>Jews. Appearently, the Cursades(a religious war agianst the muslilams\n>in \'the holy land\') sparked the widespread persecution of muslilams \n>and jews in europe. Among the supporters of the persiecution, were none \n>other than Martin Luther, and the Vatican.\n>\n>Later, Hitler would use Luthers writings to justify his own treatment\n>of the jews.\n>> Genocide is Caused by Theism : Evidence?\n\nHeck, I remember reading a quote of Luther as something like: "Jews should\nbe shot like deer."  And of course much Catholic doctrine for centuries was \nextremely anti-Semitic.\n\n\n\n-- \n"Are you so sure that your truth and your justice are worth more than the\ntruth

In [20]:
c.predict_proba

<function sklearn.pipeline.Pipeline.predict_proba(self, X)>

In [22]:
type(c.predict_proba)

function